In [5]:
#check  point 3.3 #left over will not be shown in camera and also the total transparent
import cv2
import numpy as np
import os
import time
# Load pre-trained face detection model
prototxt_path = r'.\computer_vision-master\CAFFE_DNN\deploy.prototxt'
caffemodel_path = r'.\computer_vision-master\CAFFE_DNN\res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Initialize webcam
cam = cv2.VideoCapture(0)

# Set frame width, height, and frame rate
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cam.set(cv2.CAP_PROP_FPS, 30)

# Global variables
dress_images = []

user_picture_path = r'.\assets'
total_dresses = 11
# Load dress images
def select_dress_images():
    global dress_images
    dress_images.clear()
    for i in range(total_dresses):
        image_path = os.path.join(user_picture_path, f'shirt{i}.png')
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        print(image_path)
        dress_images.append(img)

def apply_dress(frame, dress_img, x, y, w, h):
    dress_h, dress_w = dress_img.shape[:2]

    # Calculate the scale factors for resizing the dress
    scale_factor_x = w / dress_w
    scale_factor_y = h / dress_h

    # Take the maximum scale factor to enlarge the dress as much as possible while fitting within the detected face
    scale_factor = max(scale_factor_x, scale_factor_y)

    # Resize the dress image to fit the detected face
    resized_dress = cv2.resize(dress_img, None, fx=scale_factor * 1.5, fy=scale_factor * 1.5)

    # Calculate the coordinates for the dress overlay
    dress_y = y + int(h * 0.8)  # Adjust the value to position the dress below the neck
    dress_x = x - int((resized_dress.shape[1] - w) / 2) - 250

    # Ensure dress overlay does not go out of bounds
    dress_y_end = min(dress_y + resized_dress.shape[0], frame.shape[0])
    dress_x_end = min(dress_x + resized_dress.shape[1], frame.shape[1])

    # Calculate the region of interest for the dress overlay
    dress_roi_y_start = max(dress_y, 0)
    dress_roi_y_end = min(dress_y_end, frame.shape[0])
    dress_roi_x_start = max(dress_x, 0)
    dress_roi_x_end = min(dress_x_end, frame.shape[1])

    # Calculate the region of interest in the dress image
    dress_img_roi_y_start = dress_roi_y_start - dress_y
    dress_img_roi_y_end = dress_img_roi_y_start + (dress_roi_y_end - dress_roi_y_start)
    dress_img_roi_x_start = dress_roi_x_start - dress_x
    dress_img_roi_x_end = dress_img_roi_x_start + (dress_roi_x_end - dress_roi_x_start)

    # Extract the dress overlay and mask within the region of interest
    dress_overlay = resized_dress[dress_img_roi_y_start:dress_img_roi_y_end, dress_img_roi_x_start:dress_img_roi_x_end]
    dress_mask = dress_overlay[:, :, -1] / 255.0

    # Invert the dress mask
    inv_dress_mask = 1.0 - dress_mask

    # Apply the dress overlay to the frame within the region of interest
    frame[dress_roi_y_start:dress_roi_y_end, dress_roi_x_start:dress_roi_x_end, :3] = (
            dress_mask[:, :, np.newaxis] * dress_overlay[:, :, :3] +
            inv_dress_mask[:, :, np.newaxis] * frame[dress_roi_y_start:dress_roi_y_end, dress_roi_x_start:dress_roi_x_end, :3]
    )



def process_and_save_output(frame, dress_index, detections, x, y, w, h):
    global user_picture_path
    if dress_images and user_picture_path:
        output_folder = 'outputs'
        os.makedirs(output_folder, exist_ok=True)
        output_path = os.path.join(output_folder, f'output{dress_index}.png')  
        frame_copy = frame.copy()
        dress_img = dress_images[dress_index]
        apply_dress(frame_copy, dress_img, x, y, w, h)
        cv2.imwrite(output_path, frame_copy)
        print(f'Saved output image: {output_path}')
        #os.system(f'start {output_path}') #if want pop up

def main():
    # Select dress images
    select_dress_images()
    
    start_time = time.time()
    current_dress_index = 0
    processed_all_dresses = False

    while cam.isOpened() and not processed_all_dresses:
        ret, frame = cam.read()
        if not ret:
            print('Error: Failed to open webcam or read frame')
            break
            
        # Convert frame to blob for face detection
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        
        # Iterate through detections
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:  # Confidence threshold
                # Get face coordinates
                box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                (x, y, w, h) = box.astype(int)
                
                # Apply dress on detected face
                if dress_images:
                    if time.time() - start_time >= 1:
                        dress_index = current_dress_index % len(dress_images)
                        process_and_save_output(frame, dress_index, detections, x, y, w, h)
                        current_dress_index += 1

                        if current_dress_index >= len(dress_images):
                            processed_all_dresses = True
                            break

        # Display frame with dress overlay
        cv2.imshow('Virtual Try-On', frame)
        
        key = cv2.waitKey(1)
        if key == ord('.'):
            break

    cv2.destroyAllWindows()
    cam.release()

if __name__ == "__main__":
    main()


.\assets\shirt0.png
.\assets\shirt1.png
.\assets\shirt2.png
.\assets\shirt3.png
.\assets\shirt4.png
.\assets\shirt5.png
.\assets\shirt6.png
.\assets\shirt7.png
.\assets\shirt8.png
.\assets\shirt9.png
.\assets\shirt10.png
Saved output image: outputs\output0.png
Saved output image: outputs\output1.png
Saved output image: outputs\output2.png
Saved output image: outputs\output3.png
Saved output image: outputs\output4.png
Saved output image: outputs\output5.png
Saved output image: outputs\output6.png
Saved output image: outputs\output7.png
Saved output image: outputs\output8.png
Saved output image: outputs\output9.png
Saved output image: outputs\output10.png
